# Prototipo de Homología de Khovanov para enlaces y marañas - Numpy

Formato de entrada:
- Enlace/maraña dado en forma de lista de cruces PD = `[(a_1,b_1,c_1,d_1),(...)]`
- $(a,b,c,d)$ indica el cruce `por debajo/undercossing` de forma `a-c por encima de b-d`
- Así, la 0-resolución en el cruce `(a,b,c,d)` conecta `a-b` y `c-d`.
- La 1-resolución conecta `a-d` y `b-c`.


Nota: esta es la versión que emplea cálculo simbólico por medio del paquete SymPy, de forma que se tiene alguna funcionalidad ampliada
(representantes de los grupos de homología, por ejemplo). Para aplicación más efectiva a nudos/marañas grandes se recomienda la versión con NumPy, que se encuentra más optimizada a la hora de hacer el cálculo de kernels e imágenes

## Paquetes

In [45]:
from fractions import Fraction
import itertools
from collections import defaultdict
import numpy as np
import math

## Maquinaria algebraica básica

In [48]:
# ---------- Bases ----------
V_base = ['v-', 'v+']  # Base de V
W_base = ['w'] # Base de W
deg = {'v-': -1, 'v+': 1, 'w': -1}

def tensor_base(l): #Entrada de 0s y 1s denotando el espacio producto tensorial. 0-> círculo, 1-> arco (W)
    factor_bases = []
    for x in l:
        if x == 0:
            factor_bases.append(V_base)
        elif x == 1:
            factor_bases.append(W_base)
        else:
            raise ValueError("tensor_base: Entrada debe tener 0's y 1's")

    return [tuple(factors) for factors in itertools.product(*factor_bases)] #Base ordenada natural

def tensor_grado_vec(vec): #graduación sobre el producto tensorial
    return sum(deg[v] for v in vec)

In [50]:
# ---------- Operaciones algebraicas  ----------

def mult(a, b): #Fusión de círculos V x V -> V
    if a == 'v-' and b == 'v-': return None
    if a == 'v-' and b == 'v+': return 'v-'
    if a == 'v+' and b == 'v-': return 'v-'
    if a == 'v+' and b == 'v+': return 'v+'
    raise ValueError('Error elemento no en base')

def comult(a): #División de círculos V -> V x V
    if a == 'v+':
        return [(('v-', 'v+'), 1), (('v+', 'v-'), 1)]
    if a == 'v-':
        return [(('v-', 'v-'), 1)]
    raise ValueError('Error elemento no en base')
    
def nul(a): #Silla (0:W x W -> W x W)
    return None

def tanmult(a,b): #Unión de círculo y arco W x V -> W
    if (a == 'w' and b == 'v-') or (a == 'v-' and b == 'w'): return None
    if (a == 'w' and b == 'v+') or (a == 'v+' and b == 'w'): return 'w'
    raise ValueError('Multiplicación Arco-Círculo no válida')

def tancomult(w,di='i'): #Separación de arco en círculo y arco (especificando dirección, en versión actual sólo hace falta izda)
    if di=='d': return [(('w', 'v-'), 1)]
    if di=='i': return [(('v-', 'w'), 1)]

## Construcción de matrices

In [53]:
def matriz_merge(l_orig, pos): #Construye la martriz para la fusión de círculos en el espacio l_orig entre la componente pos y pos+1 (deben ser V's)
    if not (l_orig[pos]==0 and l_orig[pos+1]==0):
        raise ValueError('Merge debe ser V-V')
    base_orig = tensor_base(l_orig)
    base_dest = tensor_base(l_orig[:pos]+l_orig[pos+1:])
    M = np.zeros((len(base_dest), len(base_orig)))

    for col, src in enumerate(base_orig): #Para cada columna se toman los elementos correspondientes de la base ordenada,                         #se aplica 
        a = src[pos]                      # se aplica la operación correspondiente y se mete un 1 en la posición que corresponde
        b = src[pos + 1]                  # a la imagen obtenida en la base destino.
        prod = mult(a, b)
        if prod is None:
            continue
        tgt = src[:pos] + (prod,) + src[pos+2:]
        try:
            fil = base_dest.index(tgt)
        except ValueError:
            raise ValueError(f"{tgt} no está en base_dest={base_dest}")
        M[fil, col] = 1
    return M

def matriz_split(l_orig, pos): #Mismo proceso para el resto de operaciones
    if not (l_orig[pos]==0):
        raise ValueError('Split debe ser sobre V')
    base_orig = tensor_base(l_orig)
    base_dest = tensor_base(l_orig[:pos+1]+[0]+l_orig[pos+1:]) #Aparece un círculo (0)
    M = np.zeros((len(base_dest), len(base_orig)))
    for j, src in enumerate(base_orig):
        a = src[pos]
        resultado = comult(a)
        for tupla, coef in resultado:
            tgt = src[:pos] + tupla + src[pos+1:] #se crea el vector añadiendo la imagen 'local' en la posición
            i = base_dest.index(tgt)              # se busca elíndice en el ordenamiento del codominio de la imagen
            M[i, j] += coef         # se añade un 1 donde corresponda
    return M

def matriz_tanmerge(l_orig, pos):                # Matriz unión de arco y círculo, se detecta la dirección automáticamente
    if (l_orig[pos]==1 and l_orig[pos+1]==0):
        direc='d'
    elif (l_orig[pos]==0 and l_orig[pos+1]==1):
        direc='i'
    else:
        raise ValueError('Merge debe ser V-W')
    base_orig = tensor_base(l_orig)
    if direc=='d':
        base_dest = tensor_base(l_orig[:pos+1]+l_orig[pos+2:]) #Desaparece un círculo en la posición correspondiente
    elif direc=='i':
        base_dest = tensor_base(l_orig[:pos]+l_orig[pos+1:])
        
    M = np.zeros((len(base_dest), len(base_orig)))

    for col, src in enumerate(base_orig): #Se repite el proceso análogo a la fusión de círculos
        a = src[pos]
        b = src[pos + 1]
        prod = tanmult(a, b)
        if prod is None:
            continue
        tgt = src[:pos] + (prod,) + src[pos+2:]
        try:
            fil = base_dest.index(tgt)
        except ValueError:
            raise ValueError(f"{tgt} no está en `base_dest={base_dest}")
        M[fil, col] = 1
    return M

def matriz_tansplit(l_orig, pos, direc='i'): #Separación en arco y círculo, análogo a separación de círculos
    if not (l_orig[pos]==1):
        raise ValueError('Tansplit debe ser sobre W')
    base_orig = tensor_base(l_orig)
    if direc=='d':
        base_dest = tensor_base(l_orig[:pos]+[1,0]+l_orig[pos+1:])
    if direc=='i':
        base_dest = tensor_base(l_orig[:pos]+[0,1]+l_orig[pos+1:])
    M = np.zeros((len(base_dest), len(base_orig)))
    for j, src in enumerate(base_orig):
        a = src[pos]
        resultado = tancomult(a,direc)
        for tupla, coef in resultado:
            tgt = src[:pos] + tupla + src[pos+1:]
            i = base_dest.index(tgt)
            M[i, j] += coef
    return M

def matriz_nul(l_orig): #Sólo crea la matriz nula de longitud pertinente
    base_orig=tensor_base(l_orig)
    base_dest=base_orig
    return np.zeros((len(base_dest),len(base_orig)))

## Encaje de matrices en posición

In [56]:
# Identidad en espacio l, actualmente en desuso
def tensor_id(l):
    if len(l) == 0:
        raise ValueError("l debe ser no vacía")
    return np.eye(2**(len(l)-sum(l))+sum(l))

# Función Encaja la matriz local, en la que siempre aparecen/desaparecen componentes adyacentes en el ordenamiento, en el caso
# más general de componentes posiblemente no adyacentes. Por ejemplo, la matriz de la fusión de los componentes 1 y 5
# se obtendría encajando la matriz fusión con pos_orig=[1,5] y pos_dest la deseada. Patron_x_local es una lista de 0's y 1's
# especificando sobre qué componentes (W ó V) origen y destino actúa la matriz local. Lo pasaremos como argumento, aunque
# patron_orig_local se puede deducir de l_total en las posiciones en las que actúa.
def encaje_matrizlocal(l_total, local_map, pos_orig, pos_dest, patron_orig_local=None, patron_dest_local=None):
    n_total = len(l_total)
    if any(not (0 <= p < n_total) for p in pos_orig):
        raise ValueError("pos_orig debe contener índices entre 0 y n")
    if len(set(pos_orig)) != len(pos_orig):
        raise ValueError("pos_orig no puede repetir índices")
    if len(set(pos_dest)) != len(pos_dest):
        raise ValueError("pos_dest no puede repetir índices")
    if patron_orig_local is None:
        patron_orig_local = [l_total[p] for p in pos_orig] #Cadena de 0's y 1's denotando la forma (círculos o arcos) de las posiciones origen
    if patron_dest_local is None:
        raise ValueError("se debe especificar patron_dest_local") #Se pasa como input según el tipo de cobordismo
    base_orig_local = tensor_base(patron_orig_local) 
    base_dest_local = tensor_base(patron_dest_local)
    if local_map.shape[1] != len(base_orig_local) or local_map.shape[0] != len(base_dest_local): #No debería pasar, control
        raise ValueError(
            f"dimensiones de local_map {local_map.shape[0]}x{local_map.shape[1]} no encajan con "
            f"base_orig_local {base_orig_local} (cols {len(base_orig_local)}) y/o "
            f"base_dest_local {base_dest_local} (rows {len(base_dest_local)})"
        )

    patron_dest_completo = [] #Se debe recuperar a partir del patrón original, quitando las posiciones origen y añadiendo las destino,
    desp=0                    # todo ello con cuidado de que las posiciones encajen
    t=0
    for i in range(n_total):
        if i in pos_orig and i+desp not in pos_dest:
            desp-=1
            continue          
        elif i+desp in pos_dest:
            patron_dest_completo.append(patron_dest_local[t])
            t+=1
            if i not in pos_orig:
                desp=+1
                patron_dest_completo.append(l_total[i])
        else:
            patron_dest_completo.append(l_total[i])

    if len(pos_dest)==2 and t==1: #Arreglamos manualmente si resulta que la nueva componente aparece al final
        patron_dest_completo.append(patron_dest_local[1])
                                    
    base_orig = tensor_base(l_total)
    base_dest = tensor_base(patron_dest_completo)
    M = np.zeros((len(base_dest), len(base_orig)))
    for col, tupla_orig in enumerate(base_orig):
        tupla_orig_local = tuple(tupla_orig[p] for p in pos_orig) #Tupla de v_-, v_+, w representando el vector en el producto tensorial
        try:                                                              
            col_local = base_orig_local.index(tupla_orig_local) #Sacamos el índice del elemeno en la base ordenada local de partida para aplicar local_map
        except ValueError:
            raise RuntimeError(f"Elemento local {tupla_orig_local} no encontrada en la base local de partida")
        for fil_local in range(local_map.shape[0]): #Para cada coeficiente no trivial en la columna (el elemento correspondiente aparece como sumando en la imagen)
            coef = local_map[fil_local, col_local] # se añadirá este coeficiente en el lugar "global" correspondiente
            if coef == 0:
                continue #No hace falta hacer nada
            tupla_out = base_dest_local[fil_local] #imagen "local" bajo el cobordismo dado por la matriz local
            componentes_dest = []
            desp=0
            t=0
            for i in range(n_total):
                if i in pos_orig and i+desp not in pos_dest:
                    desp-=1
                    continue
                elif i+desp in pos_dest:
                    componentes_dest.append(tupla_out[t])
                    t+=1
                    if i not in pos_orig:
                        desp+=1
                        componentes_dest.append(tupla_orig[i])
                else:
                    componentes_dest.append(tupla_orig[i])
            if len(pos_dest)==2 and t==1: 
                componentes_dest.append(tupla_out[1])
            if len(componentes_dest) != len(patron_dest_completo):
                raise RuntimeError("La imagen no tiene la longitud correcta")

            tupla_dest = tuple(componentes_dest) #Se convierte en tupla ordenada
            try:
                fil = base_dest.index(tupla_dest) #Encontramos el índice del sumando de la imagen en la base destino
            except ValueError:
                raise RuntimeError(f"Elemento imagen {tupla_dest} no está en la base del codominio {base_dest}")

            M[fil,col] += coef #Se añade el coeficiente en la posición encontrada

    return M

## Resolución de estados

In [59]:
# ---------- PD -> Resolución arcos/círculos ----------
def resolver_estado(state, PD): #Da la resolución de un diagrama PD dada la cadena "state" de 0's y 1's indicando el tipo de resolución aplicado a cada cruce
    occ = defaultdict(list) #lista ordenada de cruces, asignamos una numeración
    for ci, cr in enumerate(PD): #ci es índice del cruce, cr es el propio cruce
        for pos, lab in enumerate(cr): #lab es la etiqueta del arco involucrado y pos el índice (dentro del cruce) que le asignamos
            occ[lab].append((ci, pos)) #Se añade una entrada al diccionario "etiqueta de arco -> cruce en el que está involucrado, posición (0,1,2 ó 3) dentro de ese cruce"

    # Construcción del "grafo" como diccionario
    grafo = defaultdict(list)
    suelto = {}  # Extremos que solo aparecen una vez, deben estar fijos a la frontera (tomamos ese convenio para marañas para facilitar la notación)

    # Extremos de los arcos
    for lab, lst in occ.items(): #Recorremos etiquetas de arco y sus correspondientes cruces y posición dentro del cruce
        if len(lst) == 2: #Aparece dos veces, no está fijo a la frontera. Se debe proceder por ambos extremos.
            a, b = lst
            grafo[a].append(b); grafo[b].append(a) #añadimos el otro cruce en el que participa a las entradas del diccionario, de forma que se "conectan" los dos subgrafos
        elif len(lst) == 1: #Solo participa en un cruce; estará fijo a la frontera.
            suelto[lst[0]] = lab #Los extremos libres se juntan a un cruce ficticio 0 representando la frontera
        else:
            raise ValueError(f"Etiqueta de arco {lab} aparece {len(lst)} veces; debe ser 1 o 2")
    # Resolución del cruce; bit indica 0-resolución o 1-resolución. Se conecta a-b c-d o a-d b-c según corresponda
    for ci, cr in enumerate(PD):
        bit = int(state[ci])
        if bit == 0: #0-solución, se conecta 0-1 y 2-3 (0,1,2,3 denotan a,b,c,d; los hemos numerado con 'pos')
            p1, p2 = (ci,0), (ci,1)
            p3, p4 = (ci,2), (ci,3)
        else: #1-solución
            p1, p2 = (ci,0), (ci,3)
            p3, p4 = (ci,1), (ci,2)
        grafo[p1].append(p2); grafo[p2].append(p1) #Se conectan los grafos como antes, 
        grafo[p3].append(p4); grafo[p4].append(p3) #convirtiendo varios arcos en una sola componente (aunque sigue recogiendo la información de los arcos originales que la componen)

    visitado = set()
    circulos, arcos = [], []

    # Tras conectar los grafos, exploramos los extremos que quedan
    for ci, cr in enumerate(PD):
        for pos in range(4):
            p = (ci,pos) #Número de cruce y posición en el mismo, representa un extremo de arco
            if p in visitado: continue #No hace falta explorar
            comp = []; stack=[p]
            while stack: #Repetimos hasta que se tenga una lista vacía de extremos de arcos sin recorrer (por lo que se ha completado la componente)
                u = stack.pop() #Se saca el último de la lista
                if u in visitado: continue #Ya se ha añadido, no hace falta hacer nada
                visitado.add(u); comp.append(u) #Se añade la entrada a visitado y a la componente de la que formará parte
                for v in grafo[u]: #grafo[u] nos da a qué otros extremos se ha conectado u
                    if v not in visitado: stack.append(v) #Se añade el extremo para seguir completando la componente
            # Comprobación de cuantos extremos en la frontera hay en esta componente de la resolución
            ext = [u for u in comp if u in suelto]
            if not ext: #No hay extremos sueltos, es un círculo
                circulos.append(sorted(comp)) #Se construye un círculo como una lista (ci,pos) de cruces y posición en el cruce de los arcos del PD involucrados
            elif len(ext)==2: #Hay dos extremos fijos a la frontera, es un arco. Se procede como en el círculo, pero añadiendo los 2 extremos de arco suelto
                arcos.append((sorted(comp), [suelto[e] for e in ext]))
            else:
                raise ValueError(f"Componente con {len(ext)} extremos en la frontera, PD no válido")

    # Se ordenan los círculos y arcos. lambda comp: min(comp) indica una función que a cada componente visto como lista asigna su mínimo (es un ejemplo de lambda-cálculo)
    circulos = sorted(circulos, key=lambda comp: min(comp))
    arcos = sorted(arcos, key=lambda comp: min(comp[0]))

    return circulos, arcos

def resolver_todo(PD): #Se aplica la función anterior para todas las posibles resoluciones (states recorre {0,1}^n con n el número de cruces)
    n = len(PD)
    states = [''.join(bits) for bits in itertools.product('01', repeat=n)]
    return {s: resolver_estado(s, PD) for s in states} #Devolvemos un conjunto con todas las resoluciones

#Construimos una función que, dado el PD y las cadenas de 0's y 1's de dos de sus resoluciones, identifique qué cobordismo generador los conecta
def identificar_cobord(PD, s, s2):
    difs = [i for i,(a,b) in enumerate(zip(s,s2)) if a!=b] #Posiciones (cruces) en las que difieren las cadenas
    if len(difs) != 1:
        raise ValueError('Las resoluciones no difieren en un único cruce')
    j = difs[0] #Cruce en el que difieren, obtenemos las dos resoluciones en cuestión
    circulos_s, arcos_s   = resolver_estado(s, PD)
    circulos_s2, arcos_s2 = resolver_estado(s2, PD)
    extremos_j= [(j,0),(j,1),(j,2),(j,3)] #Extremos de arco involucrados, para luego recuperar las componentes a las que pertenecen tanto en T_s como en T_s2

    #Construimos una función que encuentre las componentes adyacentes a los extremos dados
    def loc_componentes(circulos, arcos, extremos):
        componentes = {}
        for ind, comp in enumerate(circulos): #Recorremos los círculos con su índice
            for p in comp: #Recorremos los extremos de arco de la componente para ver si alguno está en nuestra lista "extremos"
                if p in extremos:
                    componentes.setdefault(('C', ind), []).append(p) #Se añade el índice del círculo junto a un indicador 'C'
        for ind, (comp, ex) in enumerate(arcos): #Lo mismo para arcos
            for p in comp:
                if p in extremos:
                    componentes.setdefault(('A', ind), []).append(p)
        return componentes
    
    map_s  = loc_componentes(circulos_s, arcos_s, extremos_j)
    map_s2 = loc_componentes(circulos_s2, arcos_s2, extremos_j)
    invol_s  = sorted(map_s.keys()) #Guardamos (las keys de) las componentes parte del cobordismo en la resolución de partida y de llegada
    invol_s2 = sorted(map_s2.keys())
    #Identificamos el tipo de cobordismo según si las componentes involucradas son círculos o arcos.
    if all(k[0]=='C' for k in invol_s) and len(invol_s)==2 and \
       all(k[0]=='C' for k in invol_s2) and len(invol_s2)==1:
        i1, i2 = invol_s
        return {'tipo':'merge', 'ind_cruce': j, 's':s, 's2':s2,
                'orig':invol_s, 'dest':invol_s2} #Devolvemos diccionario con información necesaria

    if all(k[0]=='C' for k in invol_s) and len(invol_s)==1 and \
       all(k[0]=='C' for k in invol_s2) and len(invol_s2)==2:
        return {'tipo':'split', 'ind_cruce': j, 's':s, 's2':s2,
                'orig':invol_s, 'dest':invol_s2}

    if len(invol_s)==2 and any(k[0]=='C' for k in invol_s) and any(k[0]=='A' for k in invol_s) \
       and len(invol_s2)==1 and invol_s2[0][0]=='A':
        return {'tipo':'tanmerge', 'ind_cruce': j, 's':s, 's2':s2,
                'orig':invol_s, 'dest':invol_s2}

    if len(invol_s)==1 and invol_s[0][0]=='A' and \
       len(invol_s2)==2 and any(k[0]=='A' for k in invol_s2) and any(k[0]=='C' for k in invol_s2):
        return {'tipo':'tansplit', 'ind_cruce': j, 's':s, 's2':s2,
                'orig':invol_s, 'dest':invol_s2}
    
    if len(invol_s)==2 and all(k[0]=='A' for k in invol_s) and  \
        len(invol_s2)==2 and all(k[0]=='A' for k in invol_s):
        return {'tipo':'nul', 'ind_cruce': j, 's':s, 's2':s2,
                'orig':invol_s, 'dest':invol_s2}

    #No debería suceder, sólo para identificar errores
    return {'tipo':'NV', 'ind_cruce': j, 's':s, 's2':s2,
            'orig':invol_s, 'dest':invol_s2}

def construir_transiciones(PD):
    n = len(PD); states = [''.join(bits) for bits in itertools.product('01', repeat=n)]
    transiciones = []
    for s in states: #Para cada estado cambiamos un bit en cada posición y calculamos la transición/cobordismo correspondiente
        for j in range(n): #Actualmente calcula también transiciones en 'reverso' 1->0, aunque luego no entra en el cálculo del diferencial
            flip = '1' if s[j]=='0' else '0'
            s2 = s[:j] + flip + s[j+1:]
            tr = identificar_cobord(PD, s, s2)
            tr['signo'] = (-1) ** sum(int(x) for x in s[:j]) #Incluimos una entrada en el diccionario con el signo apropiado
            transiciones.append(tr)
    return transiciones #Devuelve todas las 'flechas' del cubo (y sus reversas, en desuso)


## Construcción de cadena

In [62]:

#Función que devuelve el patrón local de destino a partir de origen y tipo (y posiblemente dirección)
# EN DESUSO - Versión actual pasa ya los patrones destino como argumento
def sacar_patrones(tr_tipo, patron_orig_local, direc='i'):
    if tr_tipo == 'merge':
        a, b = patron_orig_local
        if a == 0 and b == 0:
            return patron_orig_local, [0]    # VxV -> V
        else:
            return patron_orig_local, [1]    # No debería suceder - revisar

    if tr_tipo == 'split':
        a = patron_orig_local[0]
        if a == 0:
            return patron_orig_local, [0,0]  # V -> VxV
        else:
            if direc is None:
                return patron_orig_local, [1,0]
            return patron_orig_local, ([1,0] if direc == 'd' else [0,1])

    if tr_tipo == 'tanmerge':
        return patron_orig_local, [1] # WxV -> W ó VxW->W

    if tr_tipo == 'tansplit':
        if direc not in ('d','i'):
            raise ValueError("tansplit con dirección no admitida")
        return patron_orig_local, ([1,0] if direc == 'd' else [0,1]) #W -> WxV ó VxW según dirección
    
    if tr_tipo == 'nul': 
        return patron_orig_local, patron_orig_local #WxW -> WxW

    raise ValueError(f"Cobordismo desconocido {tr_tipo}") #Control de errores

In [64]:
# ---------- Construcción de las matrices diferenciales ----------
def matriz_diferencial(states, state_comp, transiciones, n_menos):
    alturas = {s: s.count('1') - n_menos for s in states} #Conjunto de alturas posibles, con n_- especificado de antemano
    res_h = defaultdict(list) #Diccionario con las resoluciones a cada altura
    baselocal =defaultdict(list) #Diccionario con la base del submódulo correspondiente a una resolución. El ordenamiento de la base total a una altura viene dado por esto y la ordenación de las resoluciones
    for s in states:
        res_h[alturas[s]].append(s)
    cadena_bases = {}
    comp_patrones = {}   # Contendrá entradas -> (comp_list, patron) para cada resolución s
    for h, slist in res_h.items():
        base_lista = []
        for s in slist:
            val = state_comp[s]
            if isinstance(val, tuple) and len(val) == 2:
                circulos, arcos = val
            else:
                circulos = val #En desuso, proveniente de la versión para nudos
                arcos = []

            comp_lista = [('C', i) for i in range(len(circulos))] + [('A', j) for j in range(len(arcos))] #Por convenio ordenamos poniendo primero los círculos y luego los arcos
            patron = [0] * len(circulos) + [1] * len(arcos)  # 0 -> V, 1 -> W
            comp_patrones[s] = (comp_lista, patron)
            tb = tensor_base(patron)
            for v in tb:
                base_lista.append((s, v)) #Añadimos cada elemento a la base global (altura) y local (resolución). 
                baselocal[s].append(v)    # en el caso de la global lo añadimos especificando la resolución a la que pertenece. En las otras componentes de la suma directa (las otras resoluciones) es 0.

        cadena_bases[h] = base_lista #Guardamos la base global a esa altura

    bases_alt = sorted(cadena_bases.keys()) #Todas las bases del complejo, ordenadas por su key que es la altura
    difs = {} #Se van guardando los diferenciales
    for ind, h in enumerate(bases_alt[:-1]): #REVISAR | Se recorren todas las aristas del cubo (quedándonos con las de interés)
        base_orig = cadena_bases[h] #Fijamos bases origen y destino para construir la cadena diferencial
        base_dest = cadena_bases[h+1]
        M = np.zeros((len(base_dest), len(base_orig)))
        dest_ind = {base_dest[i]: i for i in range(len(base_dest))} #Bases indexadas 0,...,n-1 con n longitud de la base
        orig_ind = {base_orig[i]: i for i in range(len(base_orig))}
        for tr in transiciones:
            s = tr['s']; s2 = tr['s2'] #Sacamos de la transición la información de las resoluciones
            if alturas.get(s) != h or alturas.get(s2) != h+1: #Sólo intervienen las aristas entre las alturas apropiadas, aquí desaparecen las reversas
                continue
            signo = tr.get('signo', 1)

            comp_list_orig, patron_orig = comp_patrones[s] #Sacamos del diccionario las componentes que intervienen
            comp_list_dest, patron_dest = comp_patrones[s2]
            pos_orig = sorted([comp_list_orig.index(cid) for cid in tr['orig']]) #Sacamos las posiciones origen y destino el patrón para la transición
            pos_dest = sorted([comp_list_dest.index(cid) for cid in tr['dest']])
            patron_orig_local = [patron_orig[pos] for pos in pos_orig]
            patron_dest_local = [patron_dest[pos] for pos in pos_dest]
            direc = None
            if tr['tipo'] == 'tansplit': #Hace falta detectar la dirección. Lo hacemos con el siguiente código.
                dest_ids = tr['dest']
                arc_pos = None; circ_pos = None
                for tid in dest_ids:
                    if tid[0] == 'A':
                        arc_pos = comp_list_dest.index(tid)
                    elif tid[0] == 'C':
                        circ_pos = comp_list_dest.index(tid)
                direc = 'i' #Provisional

            # Obtenemos el patrón destino a partir de la función previamente construida
            _, patron_dest_local2 = sacar_patrones(tr['tipo'], patron_orig_local, direc=direc)

            # Construimos la matriz del mapping local según el tipo. Fijamos la posición a 0 ya que luego el encaje en la global lo modifica si es necesario
            if tr['tipo'] == 'merge':
                local_map = matriz_merge(patron_orig_local, 0)
            elif tr['tipo'] == 'split':
                local_map = matriz_split(patron_orig_local, 0)
            elif tr['tipo'] == 'tanmerge':
                local_map = matriz_tanmerge(patron_orig_local, 0)
            elif tr['tipo'] == 'tansplit':
                local_map = matriz_tansplit(patron_orig_local, 0, direc)
            elif tr['tipo'] == 'nul':
                local_map = matriz_nul(patron_orig_local)
            else:
                # No debería pasar
                continue

            # Comprobaciones de longitud
            esperado_col = len(tensor_base(patron_orig_local))
            esperado_fil = len(tensor_base(patron_dest_local))
            if local_map.shape[1] != esperado_col or local_map.shape[0] != esperado_fil:
                raise RuntimeError(
                    f"Dimensiones de local_map {local_map.shape[0]}x{local_map.shape[1]} incompatibles con "
                    f"patron_orig_local {patron_orig_local} (columnas: {esperado_col}) and "
                    f"patron_dest_local {patron_dest_local} (rows {esperado_fil})"
                )
            G = encaje_matrizlocal(patron_orig, local_map, pos_orig, pos_dest,#Definimos la matriz (salvo orden) de la arista llamando a la función de encaje con las posiciones adecuadas
                                 patron_orig_local=patron_orig_local,
                                 patron_dest_local=patron_dest_local)
            base_orig_local = tensor_base(patron_orig_local)
            base_dest_global = tensor_base(patron_dest)
            #Sumamos a la matriz diferencial M los coeficientes no nulos teniendo en cuenta el ordenamiento de la base destino
            for col, (s_col, vec_col) in enumerate(base_orig): #Cogemos cada elemento de la base origen (en forma (s,v)) con su correspondiente índice en la base
                if s_col != s: #Pertenece a otra resolución, ya se sumará cuando se pase por ella en el bucle principal
                    continue
                try:
                    ind_orig_local = baselocal[s].index(vec_col) #Sacamos el índice del vector dentro de la base específica para la resolución (es la base que toma G también)
                except ValueError: #No debería
                    continue
                for fil_local in range(G.shape[0]): #Recorremos los vectores base del codominio local (la resolución destino de la transición en la que estamos)
                    coef = G[fil_local, ind_orig_local]
                    if coef == 0:
                        continue
                    vec_dest = base_dest_global[fil_local] #Vemos a qué vector local corresponde la fila 
                    key_dest = (s2, vec_dest) #Añadiendo la resolución se obtiene el vector global al que corresponde la fila
                    if key_dest not in dest_ind:
                        continue
                    fil = dest_ind[key_dest] #Se debe poner el coeficiente en el índice de fila del vector en el ordenamiento global destino obtenido
                    col_global = base_orig.index((s_col, vec_col)) #Recuperamos el índice del vector origen en la base origen
                    M[fil, col_global] += signo * coef #Sumamos aportación

        difs[h] = M #Matriz diferencial a altura h

    return bases_alt, cadena_bases, difs

## Cálculo de la homología

In [111]:
# ---------- Funciones de homología por cada quantum grading ----------

#Función sencilla para obtener los diferenciales de los submódulos según el quantum grading
def submatriz(M, fils, cols):
    if len(fils) == 0 or len(cols) == 0:
        return np.zeros((len(fils), len(cols)))
    return np.array([[M[i, j] for j in cols] for i in fils])

def homologia_bigrad_altura(h, cadena_bases, difs, prev_h):
    C_base = cadena_bases[h] #Base a la altura dada
    dimC = len(C_base)
    q_val = sorted({ tensor_grado_vec(vec) for (_, vec) in C_base }) #Posibles valores "base" del q.g., luego se sumará el desplazamiento {r+n_+-n_-}
    ind_q = { q: [i for i,(_,vec) in enumerate(C_base) if tensor_grado_vec(vec)==q] for q in q_val } #Creamos diccionario que para valor de q nos de los elementos de la base de ese q.g. sin desplazar
    d_h = difs.get(h, np.zeros((0, dimC))) #Guardamos la matriz diferencial a la altura de interés. El segundo argumento es control de errores, no debería entrar en acción
    d_prev = None #Necesitamos el diferencial anterior siempre que h no sea la primera altura
    if prev_h is not None:
        d_prev = difs.get(prev_h, np.zeros((0, len(cadena_bases[prev_h])))) #Tomamos la matriz anterior siempre que haya altura previa (en caso contrario no hace falta hacer el cociente para el cálculo de la homología).
    q_dict = {} #Diccionario en el que guardamos la información relevante para cada q
    H_dim_total_b = 0
    
    #ker_dim=0
    #if d_h is not None:
        #ker_dim=len(d_h.nullspace()) #Dimensión del kernel total
    #im_dim=0
    #if d_prev is not None:
        #im_dim=len(d_prev.columnspace()) #Dimensión de la imagen del diferencial previo total
    #H_dim_total=ker_dim-im_dim #"Cocientamos" restando dimensiones
    
    #Ahora por quantum grading:
    for q, ind in ind_q.items(): #Recorremos los gradings y su correspondiente lista de índices de los vectores con ese grading en la base origen
        if d_h.shape[0] == 0:
            fil_ind = [] #Nada que hacer
        else:
            base_dest = cadena_bases.get(h+1, []) #Tomamos base de la cadena en h+1
            fil_ind = [i for i,(s,vec) in enumerate(base_dest) if tensor_grado_vec(vec)==q-1] #Nos quedamos solo con indices de vectores cuyo qg sea uno menor al de partida (recordemos que esto + el desplazamiento es pedir que coincidan, ver memoria)
        d_h_q = submatriz(d_h, fil_ind, ind) #Matriz diferencial de la subcadena para q
        if d_h_q.shape[0]==0 or d_h_q.shape[1]==0: #No hay elementos de este qg
            ker_dim_q = len(ind) #Todo es el kernel
        else:
            ker_dim_q = len(ind)-np.linalg.matrix_rank(d_h_q)
        if d_prev is not None and d_prev.shape[0]>0 and d_prev.shape[1]>0: #Cuando la matriz anterior es no trivial:
            d_prev_q = submatriz(d_prev, ind, list(range(d_prev.shape[1]))) #Matriz diferencial previa de la subcadena
            im_dim_q = np.linalg.matrix_rank(d_prev_q)
        else:
            im_dim_q=0
        if ker_dim_q == 0:
            q_dict[q] = (0, []) #kernel trivial, marcamos dimensión 0
            continue
        Hq_dim = ker_dim_q-im_dim_q #Dimensión del submódulo (h,q) salvo desplazamiento - en esta versión no hay control, debería ser positivo siempre
        print('r=',h,' q=',q, ' dim=', Hq_dim)
        H_dim_total_b += Hq_dim #Sumamos las contribuciones a la dimensión total de cada qg, debería coincidir con H_dim_total
        q_dict[q] = (Hq_dim, [])
        
    #if H_dim_total!=H_dim_total_b: #Control de errores para testeo de construcción; la dimensión homológica total debería ser la suma de las subdimensiones por qg
        #print(f"Dimensiones homológicas a altura ",{h}, "incoherentes; ",{H_dim_total}, " vs ", {H_dim_total_b})
        
    return H_dim_total_b, q_dict

#Función que ejecuta de una vez el cálculo para todas las alturas
def homologia_bigrad_todo(alturas, cadena_bases, difs):
    res = {}
    prev_h = None
    for h in alturas:
        H_total, q_info = homologia_bigrad_altura(h, cadena_bases, difs, prev_h) #Se ejecuta la función anterior y se guarda la dimensión total y la información específica por qg
        res[h] = (H_total, q_info)
        prev_h = h #Se actualiza la altura - en desuso
    return res

In [113]:
# ---------- Función para ejecutar cálculo completo ----------
def ejecutar_homologia(PD, n_menos=0, detalle=True):
    state_comp = resolver_todo(PD)
    transiciones = construir_transiciones(PD)
    states = list(state_comp.keys())
    alturas, cadena_bases, difs = matriz_diferencial(states, state_comp, transiciones, n_menos)
    res = homologia_bigrad_todo(alturas, cadena_bases, difs)
    if detalle:
        q_set=set()
        q_tabla={}
        print('Alturas:', alturas)
        for h in alturas:
            print(' r =', h, ' tamaño total de base =', len(cadena_bases[h]))
        print('\nHomología bigraduada:')
        for h, (Hdim, qdict) in res.items():
            print('\nAltura homológica', h, ' dim H^(',h,') =', Hdim)
            for q, (dim, reps) in sorted(qdict.items(), key=lambda x: x[0]): #El lambda: x->x(0) nos ordena por qg
                qaj=q+h+len(PD)-2*n_menos #Aquí es donde se introduce el ajuste por desplazamiento
                q_set.add(qaj)
                q_tabla[(h,qaj)]=int(dim)
                print('  qg =',qaj, ' dim H^(',h,',',qaj,') : ',dim)
                if len(reps) > 0:
                    print('   Vectores representantes:')
                    for v in reps:
                        terms = []
                        for i, val in enumerate(v):
                            if val != 0:
                                state, vec = cadena_bases[h][i]
                                terms.append(f"({val})*[{state}] | {vec}")
                        print('    ', ' + '.join(terms) if terms else '    0')

        #Tabla output para visualizar la información mejor
        tabla={}
        q_lista = sorted(q_set, reverse=True)
        for qg in q_lista:
            fil = {}
            for h in alturas:
                fil[h] = q_tabla.get((h, qg), 0)
            tabla[qg] = fil
        if not q_lista:
            print("  Sin tabla")
        else:
            header = ["j \\ r"] + [str(h) for h in alturas]
            col_ancho = [max(len(header[0]), 6)]
            for h in alturas:
                col_ancho.append(max(len(str(h)), 6))
            def pad(s, w): return str(s).rjust(w)
            header_line = " | ".join(pad(header[i], col_ancho[i]) for i in range(len(header)))
            sep_line = "-+-".join("-" * col_ancho[i] for i in range(len(header)))
            print()
            print(header_line)
            print(sep_line)
            for qg in q_lista:
                fil_val = [tabla[qg][h] for h in alturas]
                fil_linea = " | ".join(pad(qg, col_ancho[0]) if i == 0 else pad(fil_val[i-1], col_ancho[i])
                                      for i in range(len(header)))
                print(fil_linea)
            print()
    return {'state_comp': state_comp, 'transiciones': transiciones, 'alturas': alturas, 'resultado': res}

# Ejemplos

Nudo test 1 - isótopo a arco y circo disjuntos

In [117]:
TE=[(1,2,3,4),(2,5,6,3),(6,5,7,8),(8,7,9,4)]
_,_,_,_=ejecutar_homologia(TE,2)

r= -1  q= -3  dim= 0
r= -1  q= -1  dim= 0
r= 0  q= -4  dim= 0
r= 0  q= -2  dim= 1
r= 0  q= 0  dim= 1
r= 1  q= -3  dim= 0
r= 1  q= -1  dim= 0
r= 1  q= 1  dim= 0
r= 2  q= -2  dim= 0
r= 2  q= 0  dim= 0
Alturas: [-2, -1, 0, 1, 2]
 r = -2  tamaño total de base = 2
 r = -1  tamaño total de base = 10
 r = 0  tamaño total de base = 18
 r = 1  tamaño total de base = 10
 r = 2  tamaño total de base = 2

Homología bigraduada:

Altura homológica -2  dim H^( -2 ) = 0
  qg = -4  dim H^( -2 , -4 ) :  0
  qg = -2  dim H^( -2 , -2 ) :  0

Altura homológica -1  dim H^( -1 ) = 0
  qg = -4  dim H^( -1 , -4 ) :  0
  qg = -2  dim H^( -1 , -2 ) :  0
  qg = 0  dim H^( -1 , 0 ) :  0

Altura homológica 0  dim H^( 0 ) = 2
  qg = -4  dim H^( 0 , -4 ) :  0
  qg = -2  dim H^( 0 , -2 ) :  1
  qg = 0  dim H^( 0 , 0 ) :  1
  qg = 2  dim H^( 0 , 2 ) :  0

Altura homológica 1  dim H^( 1 ) = 0
  qg = -2  dim H^( 1 , -2 ) :  0
  qg = 0  dim H^( 1 , 0 ) :  0
  qg = 2  dim H^( 1 , 2 ) :  0

Altura homológica 2  dim H^( 2 ) 

En efecto, se obtiene la misma homología que en una cadena con única altura no trivial W x V

Nudo 'trébol' (trefoil) a izquierdas (tiene 3 cruces negativos)

In [121]:
trefoil_pd=[(1,2,3,4),(4,3,5,6),(6,5,2,1)]
_,_,_,_=ejecutar_homologia(trefoil_pd, 3)

r= -3  q= -3  dim= 1
r= -2  q= -2  dim= 0
r= -2  q= 0  dim= 1
r= -2  q= 2  dim= 0
r= -1  q= -1  dim= 0
r= -1  q= 1  dim= 0
r= 0  q= -2  dim= 0
r= 0  q= 0  dim= 1
r= 0  q= 2  dim= 1
Alturas: [-3, -2, -1, 0]
 r = -3  tamaño total de base = 8
 r = -2  tamaño total de base = 12
 r = -1  tamaño total de base = 6
 r = 0  tamaño total de base = 4

Homología bigraduada:

Altura homológica -3  dim H^( -3 ) = 1
  qg = -9  dim H^( -3 , -9 ) :  1
  qg = -7  dim H^( -3 , -7 ) :  0
  qg = -5  dim H^( -3 , -5 ) :  0
  qg = -3  dim H^( -3 , -3 ) :  0

Altura homológica -2  dim H^( -2 ) = 1
  qg = -7  dim H^( -2 , -7 ) :  0
  qg = -5  dim H^( -2 , -5 ) :  1
  qg = -3  dim H^( -2 , -3 ) :  0

Altura homológica -1  dim H^( -1 ) = 0
  qg = -5  dim H^( -1 , -5 ) :  0
  qg = -3  dim H^( -1 , -3 ) :  0

Altura homológica 0  dim H^( 0 ) = 2
  qg = -5  dim H^( 0 , -5 ) :  0
  qg = -3  dim H^( 0 , -3 ) :  1
  qg = -1  dim H^( 0 , -1 ) :  1

 j \ r |     -3 |     -2 |     -1 |      0
-------+--------+--------+--

Unknot/nudo trivial (diagrama con un cruce resultante de aplicar R1 inverso, debido a que no se puede pasar un diagrama sin cruces por el programa, al haber ambigüedad respecto a los componentes)

In [124]:
unkPD=[(1,2,2,1)]
_,_,_,_=ejecutar_homologia(unkPD, 1)

r= 0  q= -2  dim= 0
r= 0  q= 0  dim= 1
r= 0  q= 2  dim= 1
Alturas: [-1, 0]
 r = -1  tamaño total de base = 2
 r = 0  tamaño total de base = 4

Homología bigraduada:

Altura homológica -1  dim H^( -1 ) = 0
  qg = -3  dim H^( -1 , -3 ) :  0
  qg = -1  dim H^( -1 , -1 ) :  0

Altura homológica 0  dim H^( 0 ) = 2
  qg = -3  dim H^( 0 , -3 ) :  0
  qg = -1  dim H^( 0 , -1 ) :  1
  qg = 1  dim H^( 0 , 1 ) :  1

 j \ r |     -1 |      0
-------+--------+-------
     1 |      0 |      1
    -1 |      0 |      1
    -3 |      0 |      0



Maraña píldora ($T_p$ en la memoria)

In [127]:
PIL=[(1,2,3,4),(2,1,5,3)]
pildora,_,_,_=ejecutar_homologia(PIL, 0)

r= 0  q= -2  dim= 1
r= 1  q= -1  dim= 0
r= 2  q= -2  dim= 0
r= 2  q= 0  dim= 1
Alturas: [0, 1, 2]
 r = 0  tamaño total de base = 2
 r = 1  tamaño total de base = 2
 r = 2  tamaño total de base = 2

Homología bigraduada:

Altura homológica 0  dim H^( 0 ) = 1
  qg = 0  dim H^( 0 , 0 ) :  1
  qg = 2  dim H^( 0 , 2 ) :  0

Altura homológica 1  dim H^( 1 ) = 0
  qg = 2  dim H^( 1 , 2 ) :  0

Altura homológica 2  dim H^( 2 ) = 1
  qg = 2  dim H^( 2 , 2 ) :  0
  qg = 4  dim H^( 2 , 4 ) :  1

 j \ r |      0 |      1 |      2
-------+--------+--------+-------
     4 |      0 |      0 |      1
     2 |      0 |      0 |      0
     0 |      1 |      0 |      0



Cruce simple de dos arcos

In [130]:
cruce=[(1,2,3,4)]
_,_,_,_=ejecutar_homologia(cruce,0)

r= 0  q= -2  dim= 1
r= 1  q= -2  dim= 1
Alturas: [0, 1]
 r = 0  tamaño total de base = 1
 r = 1  tamaño total de base = 1

Homología bigraduada:

Altura homológica 0  dim H^( 0 ) = 1
  qg = -1  dim H^( 0 , -1 ) :  1

Altura homológica 1  dim H^( 1 ) = 1
  qg = 0  dim H^( 1 , 0 ) :  1

 j \ r |      0 |      1
-------+--------+-------
     0 |      0 |      1
    -1 |      1 |      0



Trébol desatado - Coincide con el ejemplo hecho a mano en la memoria

In [133]:
trefoildes_pd=[(1,2,3,4),(5,6,7,3),(4,7,6,1)]
_,_,_,_=ejecutar_homologia(trefoildes_pd, 0)

r= 0  q= -2  dim= 1
r= 1  q= -1  dim= 0
r= 2  q= -2  dim= 0
r= 2  q= 0  dim= 1
r= 3  q= -3  dim= 0
r= 3  q= -1  dim= 0
r= 3  q= 1  dim= 1
Alturas: [0, 1, 2, 3]
 r = 0  tamaño total de base = 2
 r = 1  tamaño total de base = 3
 r = 2  tamaño total de base = 6
 r = 3  tamaño total de base = 4

Homología bigraduada:

Altura homológica 0  dim H^( 0 ) = 1
  qg = 1  dim H^( 0 , 1 ) :  1
  qg = 3  dim H^( 0 , 3 ) :  0

Altura homológica 1  dim H^( 1 ) = 0
  qg = 3  dim H^( 1 , 3 ) :  0

Altura homológica 2  dim H^( 2 ) = 1
  qg = 3  dim H^( 2 , 3 ) :  0
  qg = 5  dim H^( 2 , 5 ) :  1

Altura homológica 3  dim H^( 3 ) = 1
  qg = 3  dim H^( 3 , 3 ) :  0
  qg = 5  dim H^( 3 , 5 ) :  0
  qg = 7  dim H^( 3 , 7 ) :  1

 j \ r |      0 |      1 |      2 |      3
-------+--------+--------+--------+-------
     7 |      0 |      0 |      0 |      1
     5 |      0 |      0 |      1 |      0
     3 |      0 |      0 |      0 |      0
     1 |      1 |      0 |      0 |      0



Imagen especular del anterior -trébol desatado a izquierdas (esta vez $n_-=3$)

In [136]:
trefoildes_pd_inv=[(1,2,3,4),(4,3,5,6),(6,5,2,7)]
_,_,_,_=ejecutar_homologia(trefoildes_pd_inv, 3)

r= -3  q= -3  dim= 1
r= -2  q= -2  dim= 1
r= -2  q= 0  dim= 0
r= -1  q= -1  dim= 0
r= 0  q= -2  dim= 0
r= 0  q= 0  dim= 1
Alturas: [-3, -2, -1, 0]
 r = -3  tamaño total de base = 4
 r = -2  tamaño total de base = 6
 r = -1  tamaño total de base = 3
 r = 0  tamaño total de base = 2

Homología bigraduada:

Altura homológica -3  dim H^( -3 ) = 1
  qg = -9  dim H^( -3 , -9 ) :  1
  qg = -7  dim H^( -3 , -7 ) :  0
  qg = -5  dim H^( -3 , -5 ) :  0

Altura homológica -2  dim H^( -2 ) = 1
  qg = -7  dim H^( -2 , -7 ) :  1
  qg = -5  dim H^( -2 , -5 ) :  0

Altura homológica -1  dim H^( -1 ) = 0
  qg = -5  dim H^( -1 , -5 ) :  0

Altura homológica 0  dim H^( 0 ) = 1
  qg = -5  dim H^( 0 , -5 ) :  0
  qg = -3  dim H^( 0 , -3 ) :  1

 j \ r |     -3 |     -2 |     -1 |      0
-------+--------+--------+--------+-------
    -3 |      0 |      0 |      0 |      1
    -5 |      0 |      0 |      0 |      0
    -7 |      0 |      1 |      0 |      0
    -9 |      1 |      0 |      0 |      0



Otro ejemplo más enrevesado, también isotópico a un arco y círculo disjuntos

In [139]:
PDT2=[(1,2,3,4),(5,6,2,1),(3,6,7,10),(7,5,8,11),(11,8,9,10)]
_,_,_,_=ejecutar_homologia(PDT2,2)

r= -1  q= -4  dim= 0
r= -1  q= -2  dim= 0
r= -1  q= 0  dim= 0
r= 0  q= -3  dim= 1
r= 0  q= -1  dim= 1
r= 0  q= 1  dim= 0
r= 1  q= -2  dim= 0
r= 1  q= 0  dim= 0
r= 2  q= -3  dim= 0
r= 2  q= -1  dim= 0
r= 3  q= -2  dim= 0
r= 3  q= 0  dim= 0
Alturas: [-2, -1, 0, 1, 2, 3]
 r = -2  tamaño total de base = 4
 r = -1  tamaño total de base = 16
 r = 0  tamaño total de base = 28
 r = 1  tamaño total de base = 20
 r = 2  tamaño total de base = 8
 r = 3  tamaño total de base = 2

Homología bigraduada:

Altura homológica -2  dim H^( -2 ) = 0
  qg = -4  dim H^( -2 , -4 ) :  0
  qg = -2  dim H^( -2 , -2 ) :  0
  qg = 0  dim H^( -2 , 0 ) :  0

Altura homológica -1  dim H^( -1 ) = 0
  qg = -4  dim H^( -1 , -4 ) :  0
  qg = -2  dim H^( -1 , -2 ) :  0
  qg = 0  dim H^( -1 , 0 ) :  0
  qg = 2  dim H^( -1 , 2 ) :  0

Altura homológica 0  dim H^( 0 ) = 2
  qg = -2  dim H^( 0 , -2 ) :  1
  qg = 0  dim H^( 0 , 0 ) :  1
  qg = 2  dim H^( 0 , 2 ) :  0

Altura homológica 1  dim H^( 1 ) = 0
  qg = 0  dim H^( 1 , 

Nudo en 8

In [142]:
f8PD=[(1,2,3,4),(2,1,6,5),(8,6,4,7),(7,3,5,8)]
_,_,_,_=ejecutar_homologia(f8PD,2)

r= -2  q= -3  dim= 1
r= -1  q= -2  dim= 0
r= -1  q= 0  dim= 1
r= -1  q= 2  dim= 0
r= 0  q= -3  dim= 0
r= 0  q= -1  dim= 1
r= 0  q= 1  dim= 1
r= 1  q= -2  dim= 0
r= 1  q= 0  dim= 1
r= 1  q= 2  dim= 0
r= 2  q= -3  dim= 0
r= 2  q= -1  dim= 0
r= 2  q= 1  dim= 0
r= 2  q= 3  dim= 1
Alturas: [-2, -1, 0, 1, 2]
 r = -2  tamaño total de base = 8
 r = -1  tamaño total de base = 16
 r = 0  tamaño total de base = 18
 r = 1  tamaño total de base = 16
 r = 2  tamaño total de base = 8

Homología bigraduada:

Altura homológica -2  dim H^( -2 ) = 1
  qg = -5  dim H^( -2 , -5 ) :  1
  qg = -3  dim H^( -2 , -3 ) :  0
  qg = -1  dim H^( -2 , -1 ) :  0
  qg = 1  dim H^( -2 , 1 ) :  0

Altura homológica -1  dim H^( -1 ) = 1
  qg = -3  dim H^( -1 , -3 ) :  0
  qg = -1  dim H^( -1 , -1 ) :  1
  qg = 1  dim H^( -1 , 1 ) :  0

Altura homológica 0  dim H^( 0 ) = 2
  qg = -3  dim H^( 0 , -3 ) :  0
  qg = -1  dim H^( 0 , -1 ) :  1
  qg = 1  dim H^( 0 , 1 ) :  1
  qg = 3  dim H^( 0 , 3 ) :  0

Altura homológica 1  d

Nudo en estrella ($5_1$ en la categorización de nudos primos)

In [145]:
st=[(1,2,3,4),(4,3,5,6),(6,5,7,8),(8,7,9,10),(10,9,2,1)]
_,_,_,_=ejecutar_homologia(st,5)

r= -5  q= -5  dim= 1
r= -4  q= -4  dim= 0
r= -4  q= -2  dim= 1
r= -4  q= 0  dim= 0
r= -4  q= 2  dim= 0
r= -4  q= 4  dim= 0
r= -3  q= -3  dim= 1
r= -3  q= -1  dim= 0
r= -3  q= 1  dim= 0
r= -3  q= 3  dim= 0
r= -2  q= -2  dim= 0
r= -2  q= 0  dim= 1
r= -2  q= 2  dim= 0
r= -1  q= -1  dim= 0
r= -1  q= 1  dim= 0
r= 0  q= -2  dim= 0
r= 0  q= 0  dim= 1
r= 0  q= 2  dim= 1
Alturas: [-5, -4, -3, -2, -1, 0]
 r = -5  tamaño total de base = 32
 r = -4  tamaño total de base = 80
 r = -3  tamaño total de base = 80
 r = -2  tamaño total de base = 40
 r = -1  tamaño total de base = 10
 r = 0  tamaño total de base = 4

Homología bigraduada:

Altura homológica -5  dim H^( -5 ) = 1
  qg = -15  dim H^( -5 , -15 ) :  1
  qg = -13  dim H^( -5 , -13 ) :  0
  qg = -11  dim H^( -5 , -11 ) :  0
  qg = -9  dim H^( -5 , -9 ) :  0
  qg = -7  dim H^( -5 , -7 ) :  0
  qg = -5  dim H^( -5 , -5 ) :  0

Altura homológica -4  dim H^( -4 ) = 1
  qg = -13  dim H^( -4 , -13 ) :  0
  qg = -11  dim H^( -4 , -11 ) :  1
  qg = -9

Ejemplo más complicado

In [148]:
PDT3=[(1,2,3,4),(2,5,6,3),(5,7,8,9),(7,10,11,12),(12,11,13,8),(14,13,15,16),(9,14,17,18),(19,6,18,20),(4,19,21,22),(21,20,17,16)]
_,_,_,_=ejecutar_homologia(PDT3,5)

r= -4  q= -6  dim= 0
r= -4  q= -4  dim= 0
r= -4  q= -2  dim= 0
r= -4  q= 0  dim= 0
r= -3  q= -5  dim= 1
r= -3  q= -3  dim= 0
r= -3  q= -1  dim= 0
r= -3  q= 1  dim= 0
r= -2  q= -6  dim= 0
r= -2  q= -5  dim= 0
r= -2  q= -4  dim= 2
r= -2  q= -3  dim= 0
r= -2  q= -2  dim= 0
r= -2  q= -1  dim= 0
r= -2  q= 0  dim= 0
r= -1  q= -6  dim= 0
r= -1  q= -5  dim= 0
r= -1  q= -4  dim= 0
r= -1  q= -3  dim= 2
r= -1  q= -2  dim= 0
r= -1  q= -1  dim= 0
r= -1  q= 0  dim= 0
r= -1  q= 1  dim= 0
r= 0  q= -5  dim= 0
r= 0  q= -4  dim= 0
r= 0  q= -3  dim= 0
r= 0  q= -2  dim= 3
r= 0  q= -1  dim= 0
r= 0  q= 0  dim= 0
r= 0  q= 1  dim= 0
r= 1  q= -6  dim= 0
r= 1  q= -5  dim= 0
r= 1  q= -4  dim= 0
r= 1  q= -3  dim= 0
r= 1  q= -2  dim= 0
r= 1  q= -1  dim= 2
r= 1  q= 0  dim= 0
r= 2  q= -7  dim= 0
r= 2  q= -5  dim= 0
r= 2  q= -4  dim= 0
r= 2  q= -3  dim= 0
r= 2  q= -2  dim= 0
r= 2  q= -1  dim= 0
r= 2  q= 0  dim= 2
r= 2  q= 1  dim= 0
r= 3  q= -6  dim= 0
r= 3  q= -5  dim= 0
r= 3  q= -4  dim= 0
r= 3  q= -3  dim= 0
r= 3  q